# Import dependencies

In [1]:
import pandas as pd
import re
import numpy as np

# Import dataframe

In [2]:
df = pd.read_csv("Resources/zillow_data.csv")

# Inspect data

In [3]:
df
# NOTES
# "Unnamed: 0" was a byproduct column of the extraction, that can be removed
# Charecters in price need to be removed
# Address will be a key in identifying the data, their city can be useful for analysis
# Beds, Bathrooms, and Square footage also need to be stripped of exta charecters
# Property type will need to be checked for categorical encoding

Unnamed: 0       Price  \
0             0    $453,200   
1             1    $119,000   
2             2  $1,295,000   
3             3     $65,000   
4             4    $379,000   
..          ...         ...   
792         792    $318,000   
793         793    $915,000   
794         794    $355,000   
795         795  $1,880,000   
796         796  $1,450,000   

                                               Address    Beds Bathrooms  \
0                  173-11 110th Ave, Jamaica, NY 11433   6 bds      4 ba   
1               100 W 57th St #19M, New York, NY 10019  Studio      1 ba   
2                      461 83rd St, Brooklyn, NY 11209   4 bds      3 ba   
3              920 Metcalf Ave APT 3G, Bronx, NY 10473    1 bd      1 ba   
4            303 E 57th St APT 32B, New York, NY 10022   2 bds      3 ba   
..                                                 ...     ...       ...   
792  76-32 Springfield Blvd FLOOR 143A2-2, Oakland ...   2 bds      1 ba   
793                30 Opal Ln, Staten Island, NY 10309   4 bds      4 ba   
794         1050 Hancock St APT 4C, Brooklyn, NY 11221   2 bds      1 ba   
795                  2373 205th St, Flushing, NY 11360   4 bds      5 ba   
796                  36-14 216th St, Bayside, NY 11361   5 bds      3 ba   

    Square Footage                 Property type  
0       2,168 sqft                 - Foreclosure  
1         450 sqft              - Condo for sale  
2       2,598 sqft           - For sale by owner  
3         700 sqft              - Condo for sale  
4       1,800 sqft              - Condo for sale  
..             ...                           ...  
792        -- sqft              - Condo for sale  
793     2,400 sqft              - House for sale  
794       850 sqft              - Condo for sale  
795     2,600 sqft            - New construction  
796        -- sqft  - Multi-family home for sale  

[797 rows x 7 columns]

# Data Cleaning

In [4]:
df.drop(columns="Unnamed: 0", inplace=True) #Drop byproduct column

In [5]:
cleaned1 = [i.replace("K", ",000") if "K" in i else i for i in df['Price']] # There are some K's in place of 000s in the data set, they ust be replaced
cleaned2 = [i.replace(",", "") for i in cleaned1]
cleaned3 = [i.replace("$", "") for i in cleaned2]
df['Price'] = cleaned3

In [6]:
# Now for Beds, Bathrooms, and Square Footage
for column in df[['Beds', "Bathrooms", "Square Footage"]].columns:
    df[column] = [re.sub("[^0-9]", "", string) for string in df[column]]

In [7]:
split_address = [i.split(",") for i in df['Address']]
city_address = [split_address[i][-2:] for i in range(len(split_address))]

In [8]:
zip_column = [re.sub("[^0-9]",'', i[1]) for i in city_address]
city_column = [i[0].strip() for i in city_address]

In [9]:
df.drop(columns = 'Address', inplace = True)

In [10]:
df['City'] = city_column
df['Zip Code'] = zip_column

In [11]:
df.isnull().sum()
# NOTES
# Beds had their studio label removed, that needs to be filled
# Some places did not have square footages, that needs to be filled or dropped

Price             0
Beds              0
Bathrooms         0
Square Footage    0
Property type     0
City              0
Zip Code          0
dtype: int64

In [12]:
df['Beds'] = [str(0) if not i else i for i in df['Beds']]

In [13]:
df['Square Footage'] = [np.nan if not i else i for i in df['Square Footage']]

In [14]:
# Convert to integers
df['Beds'] = pd.to_numeric(df['Beds'])
df['Bathrooms'] = pd.to_numeric(df['Bathrooms'])
df['Price'] = pd.to_numeric(df['Price'])
df['Square Footage'] = pd.to_numeric(df['Square Footage'])

In [15]:
index_names = df[ df['Beds'] > 50 ].index
  
# drop these row indexes
# from dataFrame
df.drop(index_names, inplace = True)

In [16]:
index_names = df[df['Bathrooms'].isnull()].index #Remove some Naans
df.drop(index_names, inplace = True)

In [17]:
df.isnull().sum()

Price               0
Beds                0
Bathrooms           0
Square Footage    129
Property type       0
City                0
Zip Code            0
dtype: int64

In [18]:
df.to_csv("Resources/cleaned_data.csv")